# (1) Data Description 

This data set contains data about plaicraft, containing data about what information the player used to sign up and each individual session of plaicraft that occured.

### Players CSV
This file contains data about the players/users sign-up information from plaicraft. There are 197 observations and 9 variables in this data set.

#### Variables
| Experience (character) | Subscribe (logical) | hashedEmail (character) | played_hours (double precision) | name (character) | gender (character) | age (double precision) | individualid (logical) | organizationName (logical) |
| ---------- | --------- | ----------- | ------------ | ---- | ------ | --- | ------------ | ---------------- |
| How familiar the player is with minecraft | Subscribe | Encrypted email address of the player | How many hours the player has spent playing plaicraft | Name of the player | Gender of the player | Age of the player | ID of the player | Organization that the player belongs to|

Some issues with the data are that there are no values for individualid and organizationName and therefore only consists of NA values.

### Sessions CSV

This file contains data about each individual session in plaicraft. There are 1536 observations and 5 variables in this data set.

#### Variables
| hashedEmail (character) | start_time (character) | end_time (character) | original_start_time (double precision) | original_end_time (double precision) |
| ----------- | ---------- | -------- | ------------------- | ----------------- |
| Encrypted email adress of the player | Time when player started a session | Time when player ended a session | Original start time | Original end time|

Some issues with the data are that the values in start_time and end_time contains two values per cell.


# (2) Question

# (3) Exploratory Data Analysis and Visualizatoin

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")

experience <- players |>
            ggplot(aes(x = experience, y = played_hours)) +
            geom_bar(stat = "identity") 
